# Building the environment

In [0]:
!pip install gym
!pip install ray[rllib]

In [0]:
import numpy as np

The class link is defined 

In [0]:
class link:
    """
    """
    def __init__(self, name, a0, a1):
        self.__name = name
        # self.__travel_time = 0 # the travel time of the link
        # self.__flow = 0 # the flow on the link
        self.__paths = [] # the paths using this link
        self.__a0 = a0
        self.__a1 = a1
        
    def add_path(self, path):
        self.__paths.append(path)
        
    def get_travel_time(self):
        return self.__travel_time
    
    def update_flow(self):
        self.__flow = 0
        for path in self.__paths:
            self.__flow += path.get_flow()
    
    def update_travel_time(self):
        self.update_flow()
        self.__travel_time = self.__a0 + self.__a1 * self.__flow
        
    def __str__(self):
        return self.__name

The class path is ...

In [0]:
class path:
    """
    Notes: 
     - We should add a method that "adds a car/updates the flow" to a path. 
       This should include the update_travel_time method.
     - The travel time of each path shouldn't start at 0, since each path always has some base travel time.
       Have the initial travel times of the path be when the congestion is 0. 
     - When you "update the travel time", add what the travel time of each road would be given
       the path's current flow.
       Maybe each link in the path should only have a function that calculates it's travel time
       given a flow on that link.
    """
    def __init__(self, name, links):
        self.__name = name
        # self.__travel_time = 0 # the travel time of the paths
        self.__flow = 0 # the flow on the paths
        self.__links = links # the paths using this link
        for link in links:
            link.add_path(self)
    
    def get_flow(self):
        return self.__flow
    
    def update_flow(self, f):
        self.__flow = f
    
    def add_flow(self, f):
        self.__flow += f
      
    # add the marginal cost to see if you converge to the social optimum
    def get_travel_time(self):
        return self.__travel_time
    
    def update_travel_time(self):
        self.__travel_time = 0
        for link in self.__links:
            self.__travel_time += link.get_travel_time()
            
    def __str__(self):
        return self.__name

The class network ...

In [0]:
class network:
  # add a method that gives the travel time
    def __init__(self, paths, links):
        self.paths = paths
        self.links = links
    
    def update(self):
        for link in self.links:
            link.update_travel_time()
        for path in self.paths:
            path.update_travel_time()
            
    def update_av(self, avs):
        for path in self.paths:
            path.update_flow(0)
        for av in avs:
            av.path.add_flow(av.flow)
        self.update()
        for av in avs:
            av.give_reward()
            
    def reset(self):
      for path in self.paths:
        path.update_flow(0)
      self.update()

In [0]:
ab = link('ab', 1, 1/100)
ac = link('ac', 2, 0)
bc = link('bc', 0.25, 0)
bd = link('bd', 2, 0)
cd = link('cd', 1, 1/100)

In [0]:
abcd = path('abcd', [ab, bc, cd])
abd = path('abd', [ab, bd])
acd = path('acd', [ac, cd])

In [0]:
braess = network([abd, acd, abcd], [ab, ac, bc, bd, cd])

In [0]:
braess.update()

In [0]:
#Nash equilibrium
abcd.update_flow(50)
abd.update_flow(25)
acd.update_flow(25)
braess.update()
print([path.get_travel_time() for path in braess.paths])

In [0]:
#social optimum
abcd.update_flow(0)
abd.update_flow(50)
acd.update_flow(50)
braess.update()
print([path.get_travel_time() for path in braess.paths])

In [0]:
## change this
# 1. to do Igor experiment 
class autonomous_vehicle:
    # p = 0.5
    def __init__(self, name, flow):
        self.name = name
        self.path = None
        self.reward = 0
        self.flow = flow
    
    def path_choice(self, path):
        if self.path == None:
            self.path = path
        # the following condition makes the system converges toward Nash
        # change this condition to make an faster convergence toward Nash
        p = abs(self.path.get_travel_time() - path.get_travel_time()) / self.path.get_travel_time()
        self.path = (path if np.random.rand() < p else self.path)
    
    def give_reward(self):
        # add here a NN which choose the path given the reward
        self.reward = - self.path.get_travel_time()

In [0]:
av1 = autonomous_vehicle('1', 25)
av2 = autonomous_vehicle('2', 25)
av3 = autonomous_vehicle('3', 25)
av4 = autonomous_vehicle('4', 25)

avs = [av1, av2, av3, av4]

In [0]:

braess.reset()

for i in range(100):
    best_path = [x for _,x in sorted([(path.get_travel_time(),path) for path in braess.paths],  key=lambda tup: tup[0])][0]
    #print([(str(path), path.flow, path.get_travel_time()) for path in braess.paths])
    #print(av2.p)
    for av in avs:
        av.path_choice(best_path)

    braess.update_av(avs)
    

In [0]:
print([str(av.path) for av in avs])
print([(str(path), path.get_flow(), path.get_travel_time()) for path in braess.paths])

In [0]:
# Create the roads
        ab = link('ab', 1, 1/100)
        ac = link('ac', 2, 0)
        bc = link('bc', 0.25, 0)
        bd = link('bd', 2, 0)
        cd = link('cd', 1, 1/100)
        roads = [ab, ac, bc, bd, cd]
        
        # Create the possible routes
        abcd = path('abcd', [ab, bc, cd])
        abd = path('abd', [ab, bd])
        acd = path('acd', [ac, cd])
        paths = [abd, acd, abcd]
        
        # Put the roads and the routes in a network
        town = network(paths, links)

### Environment Description
The environment


In [0]:
import ray
from gym.spaces import *
from ray.rllib.env import *

In [0]:
## Routing Environment 
class RoutingEnv(MultiAgentEnv):
    """A partially observed routing environment in which cars are choosing 
       routing paths each day.
    
    States
        Each vehicle knows: 
         - It's own previous path taken
         - Previous path's travel time
         - Communication from the other vehicles
    Actions
        Each vehicle will
         - Choose it's next path to take
         - Choose to send some communication symbol
    Rewards
        Each vehicle's goal is to minimize their own travel time and eventually
        reduce their marginal cost of their choices on each driver. 
        The cost function is: 
            (1) the amount of time on your own route 
            (2) the cost of your route choice on the other drivers 
                    (how much the travel times of other drivers has increased.
    Termination
        A rollout is terminated if the time horizon is reached.
        
    *****Note: Right now, there is no communication implemented here. 
         CHANGE LATER
    """
    
    def __init__(self, config):
        """Creates the routing environment for the RL training.
        
        config: A dict with following keys
           - network_name: Name of the road scenario.
        """
        # TO-DO: IMPLEMENT CREATE_NETWORK
        self.__network = create_network(config["network_name"]) 
        self.__n_actions = # Total number of possible actions
        self.__n_states = 3
        self.__n_vehicles = # Total number of vehicles or flows in this example.
        self.__reward_calculator = 
        self.__T = config["time_horizon"]
      
    
    @property
    def action_space(self):
        """ Defines the structure of the actions this env will give.
        
        Returns: A Discrete space of size n, where n is the number of actions.
        
        A Discrete space only allow the vehicles to output values within a 
        fixed range. The range of values will depend upon the number of paths 
        in the network. 
        
        Note: Depending on what we want the vehicles to communicate, this may 
              change. Also, do we account for the number of agents here?
        """
        return Discrete(self.__n_actions) 

    @property
    def observation_space(self):
        """ Defines the structure of the observations this env will give.
       
        Returns: A box with shape m x n, where:
          m: number of agents
          n: number of observations per agent
        """
        # Question: Not sure if the shape values are in the right order.
        return Box(shape=(self.__n_vehicles, self.__n_states), dtype=np.float32)
      
    def get_state(self, **kwargs):
        ####################################
        # specify desired state space here #
        ####################################
        return  ### FILL IN ###
    
    def reset(self):
        """Restarts the environment.
        
        Returns: A dictionary, representing the initial state
          - Key: Vehicle 
          - Value: Dictionary 
                 *Key: "Path", "Time", "Message"
                 *Value: Corresponding values
        """
        route_flows = network.reset()
        obs_dict = {}
        for veh in range(self.__n_vehicles):
          obs_dict[veh] = {"path": None, "time": -1, "message": None}
        # Set starting state
        self.__curr_state = obs_dict
        # Start count for env
        self__count = 0
        return obs_dict
    
    def step(self, actions):
        """Moves the environment forward one step by applying the actions of 
           the agents.
           
        What is the transition function in this step? 
         - NOT SURE! 
           
        Returns: 
          next_state_obs_dict: A dictionary representing the next state in the 
                               environment
          reward_dict: A dictionary of the rewards of actions taken by every 
                       vehicle
          done: A boolean representing if the simulation is finished
          info_dict: A dictionary of extra information? 
          
        """
        next_state = {}
        reward_dict = {}
        done = False
        info_dict = {}
        
        # Apply the actions of every agent at the same time
        for agent, rl_action in action.items():
            # TO-DO: IMPLEMENT ADDTOPATH
            network.addToPath(agent, rl_action) # Assumes no messages
            next_state[agent] = {"path": rl_action, 
                                 "time": None, 
                                 "message": None}
            
        # Calculate the path times and compute reward
        for agent, rl_action in action.items():
            # TO-DO: IMPLEMENT PATHTIME
            path_time = network.pathTime(rl_action)
            next_state[agent]["time"] = path_time
            # TO-DO: IMPLEMENT COMPUTE REWARD
            reward_dict[agent] = self.computer_reward(self.__curr_state[agent], 
                                                      rl_action)
        # Determine if the environment is finished
        if self.__count >= self.__T:
          done = True
        self.__count++
         
        return next_state, reward_dict, done, info_dict
        
    def compute_reward(self, state, rl_actions, **kwargs):
        """Computes the reward of each action for the current state."""
        reward = 0
        return reward

    

In [0]:
# add the gym running session
# give space for the neural network 

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def create_model(num_features,num_outputs):
    
    
    # create the placeholders
    input_ph = tf.placeholder(tf.float32, [None, num_features])
    output_ph = tf.placeholder(tf.float32, [None, num_outputs])

    # create a fully connected network layer
    hidden_layer1 = tf.contrib.layers.fully_connected(
        input_ph,                                               # input placeholder
        32,                                                     # number of hidden nodes
        weights_initializer=tf.truncated_normal_initializer(),  # initializer for the weights
        activation_fn=tf.nn.relu,                               # nonlinearity (None means linear combination)
        scope="hidden1",                                         # name of the node
        )
    # create a fully connected network layer
    hidden_layer2 = tf.contrib.layers.fully_connected(
        hidden_layer1,                                               # input placeholder
        32,                                                     # number of hidden nodes
        weights_initializer=tf.truncated_normal_initializer(),  # initializer for the weights
        activation_fn=tf.nn.relu,                               # nonlinearity (None means linear combination)
        scope="hidden2",                                         # name of the node
        )

    # create an output layer
    output_layer = tf.contrib.layers.fully_connected(
        hidden_layer2,                                           # input placeholder
        num_outputs,                                            # number of hidden nodes
        weights_initializer=tf.truncated_normal_initializer(),  # initializer for the weights
        activation_fn=None,                                     # nonlinearity (None means linear combination)
        scope="output",                                         # name of the node
        reuse=tf.AUTO_REUSE)
    
    return input_ph, output_ph, output_layer


In [0]:
# create a tensorflow session
sess = tf.Session()

# create loss
mse = tf.reduce_mean(0.5 * tf.square(output_pred - output_ph))

# create optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# initialize variables
sess.run(tf.global_variables_initializer())

# create saver to save model variables
saver = tf.train.Saver()